In [1]:
"scheme.jl" |> read |> String |> println

"Takes a string representing a scheme program, and splits it into tokens"
tokenize(text::String) = split(replace(text, "(" => " ( ", ")" => " ) "))

"Takes a vector of tokens, and parses it "
function parseTokens(tokens::Vector)
  if length(tokens) == 0 throw(error("Syntax Error - Unexpected EOF")) end
  token = popfirst!(tokens)
  if token == "("
      L = []
      while tokens[1] != ")" push!(L, parseTokens(tokens)) end
      popfirst!(tokens) # pop off ')'
      return L
  elseif token == ")" throw(error("Syntax Error - Unexpected )"))
  else return atom(token)
  end
end

"Numbers become numbers; Strings become strings; every other token is a symbol."
function atom(token)
    try return parse(BigInt, token)
    catch err
        try return parse(Float64, token)
        catch err
          if token[1] =='"' return String(token) else return Symbol(token) end
        end
    end
end

"An environment: a dict of {'var':val} pairs, with an outer Env"
createEnv(table = Dict(), outer = Miss

In [2]:
using BenchmarkTools
include("./scheme.jl")

evaluate (generic function with 1 method)

In [3]:
fact100 = """
(begin 
  (define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1))))))
  (fact 100)
)
"""
evaluate(fact100)

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [4]:
@btime evaluate(fact100)

  2.397 ms (5181 allocations: 243.74 KiB)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [5]:
jazr2 ="""
(begin
  (define threshold 1e-6)
  (define dx 1e-3)

  (define newton (lambda (f guess)
    (fixed-point (lambda (x) (- x (/ (f x) ((derive f threshold) x)))) guess)))

  (define derive (lambda (f dx)
    (lambda (x) (/ (- (f (+ x dx)) (f x)) dx))))

  (define close-enough? (lambda (x y)
    (< (abs (- x y)) threshold)))
    
  (define fixed-point (lambda (f n)
    (if (close-enough? n (f n))
        n
        (fixed-point f (f n)))))

  (define jazr (lambda (y)
    (newton (lambda (x) (- y (* x x)))  1.0)))

  (jazr 2)
)
"""

evaluate(jazr2)

1.414213562375442471084409572067767170706664949151239071270999436077521626854137

In [6]:
@btime evaluate(jazr2)

  13.442 ms (4797 allocations: 270.39 KiB)


1.414213562375442471084409572067767170706664949151239071270999436077521626854137

In [7]:
towersofhanoi3 = """
(begin
  (define move (lambda (n from to spare)
    (if 
      (= 0 n) "Done_Moving"
      (begin
        (move (- n 1) from spare to)
        (println "Move_" n "_from_" from "_to_" to)
        (move (- n 1) spare to from)
      )
    ))
)
  (move 3 "A" "B" "C")
)
"""
evaluate(towersofhanoi3)

Move_1_from_A_to_B
Move_2_from_A_to_C
Move_1_from_B_to_C
Move_3_from_A_to_B
Move_1_from_C_to_A
Move_2_from_C_to_B
Move_1_from_A_to_B


"Done_Moving"

In [8]:
deriv_x_square_plus_2x = """
(begin
  (define deriv (lambda (exp var)
    (if (constant? exp var) 0
        (if  (same-var? exp var) 1
             (if (sum? exp) (make-sum (deriv (a1 exp) var) (deriv (a2 exp) var))
                 (if (product? exp) (make-sum
                           (make-product (m1 exp) (deriv (m2 exp) var))
                           (make-product (m2 exp) (deriv (m1 exp) var)))
                     "ERROR"))))))
         
  (define atomic? (lambda (exp) (not (list? exp))))

  (define constant? (lambda (exp var) (and (atomic? exp) (not (equal? exp var)))))

  (define same-var? (lambda (exp var) (and (atomic? exp) (equal? exp var))))

  (define sum? (lambda (exp) (and (list? exp) (equal? (car exp) (quote +)))))

  (define product? (lambda (exp) (and (list? exp) (equal? (car exp) (quote *)))))

  (define make-sum (lambda (a1 a2)
    (if (equal? a1 0) a2
        (if (equal? a2 0) a1
            (list (quote +) a1 a2)))))
  
  (define make-product (lambda (m1 m2)
    (if (equal? m1 1) m2
        (if (equal? m2 1) m1
            (if (or (equal? m1 0) (equal? m2 0)) 0
                (list (quote *) m1 m2))))))

  (define a1 (lambda (l) (car (cdr l))))

  (define a2 (lambda (l) (car (cdr (cdr l)))))

  (define m1 a1)

  (define m2 a2)

  (deriv (quote (+ (* x x) (* 2 x))) (quote x))
)
"""
evaluate(deriv_x_square_plus_2x)

3-element Vector{Any}:
  :+
  [:+, :x, :x]
 2

In [9]:
@btime evaluate(deriv_x_square_plus_2x)

  29.568 ms (6980 allocations: 351.52 KiB)


3-element Vector{Any}:
  :+
  [:+, :x, :x]
 2

In [10]:
fibs10 = """
(begin
  (define fib (lambda (n) (if (< n 2) 1 (+ (fib (- n 1)) (fib (- n 2))))))
  (fib 10)
)
"""
evaluate(fibs10)

89

In [11]:
@btime evaluate(fibs10)

  2.936 ms (8510 allocations: 395.88 KiB)


89